In [5]:
import gym
env  = gym.make("Taxi-v3",render_mode="rgb_array").env
env.reset()
env.render()

array([[[110, 109, 106],
        [110, 109, 106],
        [124, 122, 122],
        ...,
        [108, 111, 109],
        [108, 111, 109],
        [118, 119, 119]],

       [[110, 109, 106],
        [110, 109, 106],
        [124, 122, 122],
        ...,
        [108, 111, 109],
        [108, 111, 109],
        [118, 119, 119]],

       [[114, 116, 115],
        [114, 116, 115],
        [126, 127, 126],
        ...,
        [112, 113, 111],
        [112, 113, 111],
        [118, 117, 115]],

       ...,

       [[116, 115, 116],
        [116, 115, 116],
        [106, 107, 108],
        ...,
        [113, 115, 114],
        [113, 115, 114],
        [117, 114, 117]],

       [[116, 115, 116],
        [116, 115, 116],
        [106, 107, 108],
        ...,
        [113, 115, 114],
        [113, 115, 114],
        [117, 114, 117]],

       [[115, 112, 112],
        [115, 112, 112],
        [119, 119, 117],
        ...,
        [123, 119, 118],
        [123, 119, 118],
        [114, 114, 117]]

In [7]:
env.reset()
env.render()
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space Discrete(6)
State Space Discrete(500)


In [9]:
import numpy as np
q_table = np.zeros([env.observation_space.n,env.action_space.n])
#q_table

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [18]:
import random
from IPython.display import clear_output

# Hyperparameters
alpha = 0.1
gamma = 1.0
epsilon = 0.1

# For plotting metrics
all_epochs = []
all_penalties = []
#print(env.reset())
#print(env.step(action))

(468, -1, False, False, {'prob': 1.0, 'action_mask': array([0, 1, 1, 0, 0, 0], dtype=int8)})


In [25]:
for i in range(1, 100001):
    state = env.reset()
    state = state[0]

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() # Explore action space
        else:
            action = np.argmax(q_table[state]) # Exploit learned values

        next_state, reward, done, truncated,info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")
        
print("Training finished.\n")

Episode: 100000
Training finished.



In [26]:
"""Evaluate agent's performance after Q-learning"""

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    state = state[0]
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, truncated,info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.95
Average penalties per episode: 0.0


In [32]:
print(q_table)

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 9.00000000e+00  1.00000000e+01  9.00000000e+00  1.00000000e+01
   1.10000000e+01  1.00000000e+00]
 [ 1.30000000e+01  1.40000000e+01  1.30000000e+01  1.40000000e+01
   1.50000000e+01  5.00000000e+00]
 ...
 [ 8.75025881e+00  1.60000000e+01  8.77956349e+00  1.17354669e+01
   2.21116535e+00  2.98636572e+00]
 [ 7.46651982e-01  1.20000000e+01  6.08623050e+00  8.25135424e+00
  -1.13585415e+00 -1.59830459e+00]
 [ 1.07763784e+07  1.07368939e+07  1.07725054e+07  1.07825267e+07
   1.07687981e+07  1.07724340e+07]]


In [36]:
np.savetxt("output.csv", q_table, delimiter=",", fmt='%d')